In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

#from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

In [3]:
data = pd.read_csv('F:/web mining/webmining project/crawler/annotation_data/Iphone_plus_samsung_annotated_24000.csv')
# Keeping only the neccessary columns
data = data[['text','polarity']]

In [4]:
#data

In [5]:
# Splitting the dataset into train and test set
train, test = train_test_split(data,test_size = 0.1)

# Removing neutral sentiments
    #train = train[train.sentiment != "Neutral"]

In [6]:
train_pos = train[ train['polarity'] == 'positive']
train_pos = train_pos['text']
train_neg = train[ train['polarity'] == 'negative']
train_neg = train_neg['text']
train_nut = train[ train['polarity'] == 'neutral']
train_nut = train_nut['text']

In [7]:
tweets = []
stopwords_set = set(stopwords.words("english"))

for index, row in train.iterrows():
    words_filtered = [e.lower() for e in row.text.split() if len(e) >= 3]
    words_cleaned = [word for word in words_filtered
        if 'http' not in word
        and not word.startswith('@')
        and not word.startswith('#')
        and word != 'RT']
    words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]
    tweets.append((words_cleaned,row.polarity))
    
test_pos = test[ test['polarity'] == 'positive']
test_pos = test_pos['text']
test_neg = test[ test['polarity'] == 'negative']
test_neg = test_neg['text']
test_nut = test[ test['polarity'] == 'neutral']
test_nut = test_nut['text']

In [8]:
#tweets

In [9]:
# Extracting word features
def get_words_in_tweets(tweets):
    all = []
    for (words, polarity) in tweets:
        all.extend(words)
    return all
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    features = wordlist.keys()
    return features
w_features = get_word_features(get_words_in_tweets(tweets))

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in w_features:
        features['containts(%s)' % word] = (word in document_words)
    return features


In [10]:
# Training the Naive Bayes classifier
training_set = nltk.classify.apply_features(extract_features,tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [11]:
neg_cnt = 0
pos_cnt = 0
nut_cnt = 0
for obj in test_neg: 
    res =  classifier.classify(extract_features(obj.split()))
    if(res == 'negative'): 
        neg_cnt = neg_cnt + 1
for obj in test_pos: 
    res =  classifier.classify(extract_features(obj.split()))
    if(res == 'positive'): 
        pos_cnt = pos_cnt + 1
for obj in test_nut: 
    res =  classifier.classify(extract_features(obj.split()))
    if(res == 'neutral'): 
        nut_cnt = nut_cnt + 1
        
 

[Negative]: 817/667 
[Positive]: 549/164 
[Neutral]: 1073/767 


In [20]:
print('[Negative]: %s/%s '  % (len(test_neg),neg_cnt))        
print('[Positive]: %s/%s '  % (len(test_pos),pos_cnt)) 
print('[Neutral]: %s/%s '  % (len(test_nut),nut_cnt))


a_neg = len(test_neg)
a_pos = len(test_pos)
a_nut = len(test_nut)

b_neg = neg_cnt
b_pos = pos_cnt
b_nut = nut_cnt


c_neg = b_neg/a_neg
c_pos = b_pos/a_pos
c_nut = b_nut/a_nut

print('[Negative]: %s '  % c_neg)        
print('[Positive]: %s '  % c_pos) 
print('[Neutral]: %s '  % c_nut ) 

[Negative]: 817/667 
[Positive]: 549/164 
[Neutral]: 1073/767 
[Negative]: 0.8164014687882497 
[Positive]: 0.2987249544626594 
[Neutral]: 0.7148182665424044 
